In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

print("⚽ FEATURE ENGINEERING")
print("="*60)

# Charger les données
df = pd.read_csv('../data/raw/premier_league_matches.csv')
df['date'] = pd.to_datetime(df['date'])
print(f"✅ Loaded {len(df)} matches")
print(f"📅 From {df['date'].min().date()} to {df['date'].max().date()}")
print(f"⚽ {df['home_team'].nunique()} unique teams")
print(f"\n🎯 TARGET: Create features to predict match outcomes!")

⚽ FEATURE ENGINEERING
✅ Loaded 760 matches
📅 From 2023-08-11 to 2025-05-25
⚽ 23 unique teams

🎯 TARGET: Create features to predict match outcomes!


In [2]:
# ============================================
# FEATURE 1: RECENT FORM (Forme des 5 derniers matchs)
# ============================================

print("🔄 Calculating recent form (last 5 matches)...")

# On va calculer les points des 5 derniers matchs pour chaque équipe
# Victoire = 3 points, Nul = 1 point, Défaite = 0 points

def calculate_form(team, date, n_matches=5):
    """
    Calcule les points des N derniers matchs d'une équipe AVANT une date donnée
    """
    # Matchs de cette équipe (home ou away) AVANT cette date
    team_matches = df[
        ((df['home_team'] == team) | (df['away_team'] == team)) & 
        (df['date'] < date)
    ].sort_values('date', ascending=False).head(n_matches)
    
    if len(team_matches) == 0:
        return 0  # Début de saison, pas d'historique
    
    points = 0
    for _, match in team_matches.iterrows():
        if match['home_team'] == team:
            # Cette équipe jouait à domicile
            if match['home_score'] > match['away_score']:
                points += 3  # Victoire
            elif match['home_score'] == match['away_score']:
                points += 1  # Nul
            # Défaite = 0 points
        else:
            # Cette équipe jouait à l'extérieur
            if match['away_score'] > match['home_score']:
                points += 3  # Victoire
            elif match['away_score'] == match['home_score']:
                points += 1  # Nul
    
    return points

# Tester sur le premier match
test_match = df.iloc[50]  # Match random
print(f"\n🧪 TEST sur: {test_match['home_team']} vs {test_match['away_team']}")
print(f"Date: {test_match['date']}")

home_form = calculate_form(test_match['home_team'], test_match['date'])
away_form = calculate_form(test_match['away_team'], test_match['date'])

print(f"Home form (last 5): {home_form} points")
print(f"Away form (last 5): {away_form} points")
print(f"\n💡 Form difference: {home_form - away_form} (positive = home advantage)")

🔄 Calculating recent form (last 5 matches)...

🧪 TEST sur: Luton Town FC vs Wolverhampton Wanderers FC
Date: 2023-09-23 14:00:00+00:00
Home form (last 5): 0 points
Away form (last 5): 3 points

💡 Form difference: -3 (positive = home advantage)


In [3]:
# ============================================
# APPLIQUER LA FORME À TOUS LES MATCHS
# ============================================

print("⏳ Calculating form for all 760 matches...")
print("(This might take 30-60 seconds...)\n")

# Créer les colonnes
df['home_form'] = 0
df['away_form'] = 0

# Calculer pour chaque match
for idx, match in df.iterrows():
    df.loc[idx, 'home_form'] = calculate_form(match['home_team'], match['date'])
    df.loc[idx, 'away_form'] = calculate_form(match['away_team'], match['date'])
    
    # Progress bar simple
    if (idx + 1) % 100 == 0:
        print(f"✅ Processed {idx + 1}/760 matches...")

# Créer la feature "form difference"
df['form_diff'] = df['home_form'] - df['away_form']

print(f"\n🎉 DONE! Form calculated for all matches!")
print(f"\n📊 FORM STATISTICS:")
print(df[['home_form', 'away_form', 'form_diff']].describe())

print(f"\n👀 Sample with form:")
print(df[['home_team', 'away_team', 'home_form', 'away_form', 'form_diff']].head(10))

⏳ Calculating form for all 760 matches...
(This might take 30-60 seconds...)

✅ Processed 100/760 matches...
✅ Processed 200/760 matches...
✅ Processed 300/760 matches...
✅ Processed 400/760 matches...
✅ Processed 500/760 matches...
✅ Processed 600/760 matches...
✅ Processed 700/760 matches...

🎉 DONE! Form calculated for all matches!

📊 FORM STATISTICS:
        home_form   away_form   form_diff
count  760.000000  760.000000  760.000000
mean     6.540789    6.743421   -0.202632
std      3.721308    3.676828    5.003924
min      0.000000    0.000000  -12.000000
25%      4.000000    4.000000   -3.250000
50%      6.000000    7.000000    0.000000
75%      9.000000   10.000000    3.000000
max     15.000000   15.000000   15.000000

👀 Sample with form:
                   home_team                   away_team  home_form  \
0                 Burnley FC          Manchester City FC          0   
1                 Arsenal FC        Nottingham Forest FC          0   
2            AFC Bournemouth   

In [4]:
# Vérifier des matchs plus tard dans la saison
print("🔍 Checking form for matches later in the season:\n")

# Match 100 (environ journée 13)
print("Match 100:")
print(df.iloc[100][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Match 300 (milieu de saison)
print("Match 300:")
print(df.iloc[300][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Match 500 (fin de saison)
print("Match 500:")
print(df.iloc[500][['date', 'home_team', 'away_team', 'home_form', 'away_form', 'form_diff']])

print("\n" + "="*50 + "\n")

# Statistiques: combien de matchs ont form = 0?
zero_form = df[(df['home_form'] == 0) & (df['away_form'] == 0)]
print(f"📊 Matchs avec form=0 pour les deux équipes: {len(zero_form)}/760")
print(f"💡 C'est normal au début de saison!")

# Moyenne de la forme (excluant les 0)
non_zero = df[(df['home_form'] > 0) | (df['away_form'] > 0)]
print(f"\n📈 Stats pour matchs avec historique ({len(non_zero)} matchs):")
print(non_zero[['home_form', 'away_form', 'form_diff']].describe())

🔍 Checking form for matches later in the season:

Match 100:
date         2023-11-04 12:30:00+00:00
home_team                    Fulham FC
away_team         Manchester United FC
home_form                            5
away_form                            9
form_diff                           -4
Name: 100, dtype: object


Match 300:
date         2024-04-03 19:15:00+00:00
home_team           Manchester City FC
away_team               Aston Villa FC
home_form                           11
away_form                           10
form_diff                            1
Name: 300, dtype: object


Match 500:
date         2024-11-29 20:00:00+00:00
home_team    Brighton & Hove Albion FC
away_team               Southampton FC
home_form                           10
away_form                            3
form_diff                            7
Name: 500, dtype: object


📊 Matchs avec form=0 pour les deux équipes: 14/760
💡 C'est normal au début de saison!

📈 Stats pour matchs avec historique (746 matchs

In [5]:
# ============================================
# FEATURE 2: GOALS TRENDS (Derniers 5 matchs)
# ============================================

print("⚽ Calculating goals scored & conceded trends...\n")

def calculate_goals_stats(team, date, is_home, n_matches=5):
    """
    Calcule les buts marqués et encaissés des N derniers matchs
    
    Returns: (goals_scored, goals_conceded)
    """
    # Matchs AVANT cette date
    if is_home:
        team_matches = df[
            (df['home_team'] == team) & (df['date'] < date)
        ].sort_values('date', ascending=False).head(n_matches)
    else:
        team_matches = df[
            (df['away_team'] == team) & (df['date'] < date)
        ].sort_values('date', ascending=False).head(n_matches)
    
    if len(team_matches) == 0:
        return 0, 0
    
    goals_scored = 0
    goals_conceded = 0
    
    for _, match in team_matches.iterrows():
        if is_home:
            goals_scored += match['home_score']
            goals_conceded += match['away_score']
        else:
            goals_scored += match['away_score']
            goals_conceded += match['home_score']
    
    # Moyenne par match
    avg_scored = goals_scored / len(team_matches)
    avg_conceded = goals_conceded / len(team_matches)
    
    return avg_scored, avg_conceded

# Test rapide
test_match = df.iloc[300]  # Man City vs Aston Villa
print(f"🧪 TEST: {test_match['home_team']} vs {test_match['away_team']}")
print(f"Date: {test_match['date']}\n")

home_scored, home_conceded = calculate_goals_stats(
    test_match['home_team'], 
    test_match['date'], 
    is_home=True
)
away_scored, away_conceded = calculate_goals_stats(
    test_match['away_team'], 
    test_match['date'], 
    is_home=False
)

print(f"Man City (home) last 5 home matches:")
print(f"  ⚽ Scored: {home_scored:.2f} goals/match")
print(f"  🛡️  Conceded: {home_conceded:.2f} goals/match")

print(f"\nAston Villa (away) last 5 away matches:")
print(f"  ⚽ Scored: {away_scored:.2f} goals/match")
print(f"  🛡️  Conceded: {away_conceded:.2f} goals/match")

print(f"\n💡 Offensive advantage: {home_scored - away_scored:.2f}")
print(f"💡 Defensive advantage: {away_conceded - home_conceded:.2f}")

⚽ Calculating goals scored & conceded trends...

🧪 TEST: Manchester City FC vs Aston Villa FC
Date: 2024-04-03 19:15:00+00:00

Man City (home) last 5 home matches:
  ⚽ Scored: 1.40 goals/match
  🛡️  Conceded: 0.40 goals/match

Aston Villa (away) last 5 away matches:
  ⚽ Scored: 2.20 goals/match
  🛡️  Conceded: 0.80 goals/match

💡 Offensive advantage: -0.80
💡 Defensive advantage: 0.40


In [6]:
# ============================================
# APPLIQUER GOALS STATS À TOUS LES MATCHS
# ============================================

print("⏳ Calculating goals stats for all matches...")
print("(This will be FASTER - ~30 seconds)\n")

# Créer les colonnes
df['home_goals_scored_avg'] = 0.0
df['home_goals_conceded_avg'] = 0.0
df['away_goals_scored_avg'] = 0.0
df['away_goals_conceded_avg'] = 0.0

# Calculer pour chaque match
for idx, match in df.iterrows():
    # Home team
    h_scored, h_conceded = calculate_goals_stats(
        match['home_team'], 
        match['date'], 
        is_home=True
    )
    df.loc[idx, 'home_goals_scored_avg'] = h_scored
    df.loc[idx, 'home_goals_conceded_avg'] = h_conceded
    
    # Away team
    a_scored, a_conceded = calculate_goals_stats(
        match['away_team'], 
        match['date'], 
        is_home=False
    )
    df.loc[idx, 'away_goals_scored_avg'] = a_scored
    df.loc[idx, 'away_goals_conceded_avg'] = a_conceded
    
    if (idx + 1) % 100 == 0:
        print(f"✅ Processed {idx + 1}/760...")

# Créer features dérivées
df['offensive_strength_diff'] = df['home_goals_scored_avg'] - df['away_goals_scored_avg']
df['defensive_strength_diff'] = df['away_goals_conceded_avg'] - df['home_goals_conceded_avg']

print(f"\n🎉 DONE!\n")
print("📊 GOALS FEATURES STATISTICS:")
print(df[['home_goals_scored_avg', 'away_goals_scored_avg', 
          'offensive_strength_diff', 'defensive_strength_diff']].describe())

print(f"\n👀 Sample:")
print(df[['home_team', 'away_team', 
          'home_goals_scored_avg', 'away_goals_scored_avg',
          'offensive_strength_diff']].iloc[300:305])

⏳ Calculating goals stats for all matches...
(This will be FASTER - ~30 seconds)

✅ Processed 100/760...
✅ Processed 200/760...
✅ Processed 300/760...
✅ Processed 400/760...
✅ Processed 500/760...
✅ Processed 600/760...
✅ Processed 700/760...

🎉 DONE!

📊 GOALS FEATURES STATISTICS:
       home_goals_scored_avg  away_goals_scored_avg  offensive_strength_diff  \
count             760.000000             760.000000                760.00000   
mean                1.618575               1.413004                  0.20557   
std                 0.820154               0.693515                  0.98485   
min                 0.000000               0.000000                 -3.00000   
25%                 1.000000               1.000000                 -0.40000   
50%                 1.600000               1.400000                  0.20000   
75%                 2.200000               1.800000                  0.80000   
max                 5.000000               4.000000                  4.00000  

In [7]:
# ============================================
# TARGET VARIABLE: Match Result
# ============================================

print("🎯 Creating TARGET variable (match result)...\n")

def get_result(row):
    """
    H = Home Win
    D = Draw
    A = Away Win
    """
    if row['home_score'] > row['away_score']:
        return 'H'
    elif row['home_score'] < row['away_score']:
        return 'A'
    else:
        return 'D'

df['result'] = df.apply(get_result, axis=1)

print("✅ Target created!\n")
print("📊 DISTRIBUTION DES RÉSULTATS:")
result_counts = df['result'].value_counts()
print(result_counts)
print(f"\nPourcentages:")
print((result_counts / len(df) * 100).round(2))

print(f"\n👀 Sample avec toutes les features:")
print(df[['home_team', 'away_team', 
          'home_form', 'away_form', 'form_diff',
          'offensive_strength_diff', 'defensive_strength_diff',
          'home_score', 'away_score', 'result']].iloc[300:305])

🎯 Creating TARGET variable (match result)...

✅ Target created!

📊 DISTRIBUTION DES RÉSULTATS:
result
H    330
A    255
D    175
Name: count, dtype: int64

Pourcentages:
result
H    43.42
A    33.55
D    23.03
Name: count, dtype: float64

👀 Sample avec toutes les features:
              home_team             away_team  home_form  away_form  \
300  Manchester City FC        Aston Villa FC         11         10   
301        Liverpool FC   Sheffield United FC         13          2   
302          Chelsea FC  Manchester United FC          9          7   
303   Crystal Palace FC    Manchester City FC          5         11   
304      Aston Villa FC          Brentford FC          7          3   

     form_diff  offensive_strength_diff  defensive_strength_diff  home_score  \
300          1                     -0.8                      0.4           4   
301         11                      1.4                      0.8           3   
302          2                      0.0                    

In [8]:
# ============================================
# FEATURE 3: MARKET VALUE (GAME CHANGER!)
# ============================================

print("💰 Adding Market Value Feature \n")

# Valeurs marchandes des équipes (en millions €)
# Source: Transfermarkt Août 2023
team_values = {
    # Top teams
    'Manchester City FC': 1280,
    'Arsenal FC': 1120,
    'Liverpool FC': 1000,
    'Chelsea FC': 960,
    'Manchester United FC': 880,
    'Tottenham Hotspur FC': 820,
    'Newcastle United FC': 650,
    
    # Mid-table
    'Aston Villa FC': 490,
    'Brighton & Hove Albion FC': 420,
    'Leeds United FC': 420,          # ← 2022-23
    'West Ham United FC': 380,
    'Leicester City FC': 360,        # ← 2022-23
    'Crystal Palace FC': 280,
    'Southampton FC': 280,           # ← 2022-23
    'Wolverhampton Wanderers FC': 270,
    'Fulham FC': 265,
    'AFC Bournemouth': 230,
    'Brentford FC': 225,
    'Nottingham Forest FC': 220,
    'Everton FC': 210,
    
    # Newly promoted 2023-24
    'Luton Town FC': 85,
    'Ipswich Town FC': 95,
    'Sheffield United FC': 82,
    'Burnley FC': 78
}

print("📊 Team Values (Top 5):")
sorted_values = sorted(team_values.items(), key=lambda x: x[1], reverse=True)
for team, value in sorted_values[:5]:
    print(f"  {team}: €{value}M")

print(f"\n📊 Team Values (Bottom 5):")
for team, value in sorted_values[-5:]:
    print(f"  {team}: €{value}M")

# Ajouter au DataFrame
df['home_team_value'] = df['home_team'].map(team_values)
df['away_team_value'] = df['away_team'].map(team_values)
df['value_diff'] = df['home_team_value'] - df['away_team_value']

print(f"\n✅ Market value features added!")
print(f"\n📊 Value Difference Stats:")
print(df['value_diff'].describe())

# Exemples
print(f"\n👀 Examples:")
examples = [
    ('Liverpool FC', 'Sheffield United FC'),
    ('Manchester City FC', 'Burnley FC'),
    ('Luton Town FC', 'Arsenal FC')
]

for home, away in examples:
    match = df[(df['home_team'] == home) & (df['away_team'] == away)]
    if len(match) > 0:
        match = match.iloc[0]
        print(f"\n{home} vs {away}")
        print(f"  Value diff: €{match['value_diff']}M")
        print(f"  Result: {match['result']}")

💰 Adding Market Value Feature 

📊 Team Values (Top 5):
  Manchester City FC: €1280M
  Arsenal FC: €1120M
  Liverpool FC: €1000M
  Chelsea FC: €960M
  Manchester United FC: €880M

📊 Team Values (Bottom 5):
  Everton FC: €210M
  Ipswich Town FC: €95M
  Luton Town FC: €85M
  Sheffield United FC: €82M
  Burnley FC: €78M

✅ Market value features added!

📊 Value Difference Stats:
count     760.000000
mean        0.000000
std       522.068729
min     -1202.000000
25%      -331.250000
50%         0.000000
75%       331.250000
max      1202.000000
Name: value_diff, dtype: float64

👀 Examples:

Liverpool FC vs Sheffield United FC
  Value diff: €918M
  Result: H

Manchester City FC vs Burnley FC
  Value diff: €1202M
  Result: H

Luton Town FC vs Arsenal FC
  Value diff: €-1035M
  Result: A


In [9]:
# ============================================
# FEATURE 4: WIN RATES HOME/AWAY
# ============================================

print("🏆 Calculating Win Rates (Home/Away split)...\n")

def calculate_win_rate_home(team, date, n_matches=10):
    """
    Calcule le % de victoires à DOMICILE des N derniers matchs home
    """
    matches = df[
        (df['home_team'] == team) & 
        (df['date'] < date)
    ].sort_values('date', ascending=False).head(n_matches)
    
    if len(matches) == 0:
        return 0.5  # Default: 50% (neutre)
    
    wins = (matches['home_score'] > matches['away_score']).sum()
    return wins / len(matches)

def calculate_win_rate_away(team, date, n_matches=10):
    """
    Calcule le % de victoires à L'EXTÉRIEUR des N derniers matchs away
    """
    matches = df[
        (df['away_team'] == team) & 
        (df['date'] < date)
    ].sort_values('date', ascending=False).head(n_matches)
    
    if len(matches) == 0:
        return 0.5  # Default: 50% (neutre)
    
    wins = (matches['away_score'] > matches['home_score']).sum()
    return wins / len(matches)

# Test
test_match = df.iloc[300]
print(f"🧪 TEST: {test_match['home_team']} vs {test_match['away_team']}")
print(f"Date: {test_match['date']}\n")

home_wr = calculate_win_rate_home(test_match['home_team'], test_match['date'])
away_wr = calculate_win_rate_away(test_match['away_team'], test_match['date'])

print(f"{test_match['home_team']} (home):")
print(f"  Win rate at home: {home_wr*100:.1f}%")

print(f"\n{test_match['away_team']} (away):")
print(f"  Win rate away: {away_wr*100:.1f}%")

print(f"\n💡 Home advantage: {(home_wr - away_wr)*100:+.1f}%")

🏆 Calculating Win Rates (Home/Away split)...

🧪 TEST: Manchester City FC vs Aston Villa FC
Date: 2024-04-03 19:15:00+00:00

Manchester City FC (home):
  Win rate at home: 50.0%

Aston Villa FC (away):
  Win rate away: 50.0%

💡 Home advantage: +0.0%


In [10]:
# ============================================
# TEST WIN RATES - PLUSIEURS MATCHS
# ============================================

print("🧪 TESTING WIN RATES ON MULTIPLE MATCHES\n")
print("="*70)

# Test sur 5 matchs différents
test_indices = [100, 200, 300, 400, 500]

for idx in test_indices:
    match = df.iloc[idx]
    
    home_wr = calculate_win_rate_home(match['home_team'], match['date'])
    away_wr = calculate_win_rate_away(match['away_team'], match['date'])
    
    print(f"\n📅 Match {idx}: {match['home_team']} vs {match['away_team']}")
    print(f"Date: {match['date'].date()}")
    print(f"Actual result: {match['result']}")
    print(f"\n  {match['home_team']} (home):")
    print(f"    Win rate at home: {home_wr*100:.1f}%")
    print(f"  {match['away_team']} (away):")
    print(f"    Win rate away: {away_wr*100:.1f}%")
    print(f"  💡 Difference: {(home_wr - away_wr)*100:+.1f}%")
    
    # Prédiction basée sur win rate
    if home_wr > away_wr + 0.2:
        prediction = "H (strong)"
    elif home_wr > away_wr:
        prediction = "H (slight)"
    elif away_wr > home_wr + 0.2:
        prediction = "A (strong)"
    elif away_wr > home_wr:
        prediction = "A (slight)"
    else:
        prediction = "D (balanced)"
    
    correct = "✅" if prediction[0] == match['result'] else "❌"
    print(f"  🎯 Prediction: {prediction} {correct}")
    print("-"*70)

🧪 TESTING WIN RATES ON MULTIPLE MATCHES


📅 Match 100: Fulham FC vs Manchester United FC
Date: 2023-11-04
Actual result: A

  Fulham FC (home):
    Win rate at home: 50.0%
  Manchester United FC (away):
    Win rate away: 50.0%
  💡 Difference: +0.0%
  🎯 Prediction: D (balanced) ❌
----------------------------------------------------------------------

📅 Match 200: Newcastle United FC vs Manchester City FC
Date: 2024-01-13
Actual result: A

  Newcastle United FC (home):
    Win rate at home: 80.0%
  Manchester City FC (away):
    Win rate away: 60.0%
  💡 Difference: +20.0%
  🎯 Prediction: H (slight) ❌
----------------------------------------------------------------------

📅 Match 300: Manchester City FC vs Aston Villa FC
Date: 2024-04-03
Actual result: H

  Manchester City FC (home):
    Win rate at home: 50.0%
  Aston Villa FC (away):
    Win rate away: 50.0%
  💡 Difference: +0.0%
  🎯 Prediction: D (balanced) ❌
----------------------------------------------------------------------

📅 Ma

In [11]:
# ============================================
# APPLIQUER WIN RATES À TOUS LES MATCHS
# ============================================

print("⏳ Calculating win rates for all 760 matches...")
print("(This will take ~1-2 minutes)\n")

df['home_win_rate'] = 0.0
df['away_win_rate'] = 0.0

for idx, match in df.iterrows():
    df.loc[idx, 'home_win_rate'] = calculate_win_rate_home(match['home_team'], match['date'])
    df.loc[idx, 'away_win_rate'] = calculate_win_rate_away(match['away_team'], match['date'])
    
    if (idx + 1) % 100 == 0:
        print(f"✅ Processed {idx + 1}/760...")

df['win_rate_diff'] = df['home_win_rate'] - df['away_win_rate']

print(f"\n🎉 DONE!\n")
print("📊 WIN RATE STATISTICS:")
print(df[['home_win_rate', 'away_win_rate', 'win_rate_diff']].describe())

print(f"\n👀 Sample:")
print(df[['home_team', 'away_team', 'home_win_rate', 'away_win_rate', 'win_rate_diff']].iloc[300:305])

⏳ Calculating win rates for all 760 matches...
(This will take ~1-2 minutes)

✅ Processed 100/760...
✅ Processed 200/760...
✅ Processed 300/760...
✅ Processed 400/760...
✅ Processed 500/760...
✅ Processed 600/760...
✅ Processed 700/760...

🎉 DONE!

📊 WIN RATE STATISTICS:
       home_win_rate  away_win_rate  win_rate_diff
count     760.000000      760.00000     760.000000
mean        0.441937        0.34031       0.101626
std         0.251861        0.21174       0.313198
min         0.000000        0.00000      -1.000000
25%         0.296429        0.20000      -0.100000
50%         0.400000        0.30000       0.100000
75%         0.600000        0.50000       0.300000
max         1.000000        1.00000       1.000000

👀 Sample:
              home_team             away_team  home_win_rate  away_win_rate  \
300  Manchester City FC        Aston Villa FC            0.5            0.5   
301        Liverpool FC   Sheffield United FC            0.7            0.1   
302          Chelsea 

In [12]:
import numpy as np

print("➕ Adding extra league features (points, goal diff, recent form, rest days)...")

# Assure-toi que la date est bien en datetime
df['date'] = pd.to_datetime(df['date'])

# Trier par date pour que les cumsum/rolling soient logiques
df = df.sort_values('date').reset_index(drop=True)

# ============================================
# 1) POINTS PAR MATCH (3/1/0) POUR HOME / AWAY
# ============================================

df['home_points_match'] = np.select(
    [
        df['home_score'] > df['away_score'],   # victoire domicile
        df['home_score'] == df['away_score']   # nul
    ],
    [3, 1],
    default=0
)

df['away_points_match'] = np.select(
    [
        df['away_score'] > df['home_score'],   # victoire extérieur
        df['away_score'] == df['home_score']   # nul
    ],
    [3, 1],
    default=0
)

# ============================================
# 2) POINTS CUMULÉS AVANT LE MATCH
#    (par équipe en jouant à domicile / extérieur)
# ============================================

df['home_points_cum'] = (
    df.groupby('home_team')['home_points_match']
      .cumsum()
      .shift(1)
      .fillna(0)
)

df['away_points_cum'] = (
    df.groupby('away_team')['away_points_match']
      .cumsum()
      .shift(1)
      .fillna(0)
)

# ============================================
# 3) GOAL DIFFERENCE CUMULÉ AVANT LE MATCH
# ============================================

df['home_goal_diff_match'] = df['home_score'] - df['away_score']
df['away_goal_diff_match'] = df['away_score'] - df['home_score']

df['home_goal_diff_cum'] = (
    df.groupby('home_team')['home_goal_diff_match']
      .cumsum()
      .shift(1)
      .fillna(0)
)

df['away_goal_diff_cum'] = (
    df.groupby('away_team')['away_goal_diff_match']
      .cumsum()
      .shift(1)
      .fillna(0)
)

# ============================================
# 4) RECENT FORM COURTE : POINTS SUR LES 5 DERNIERS MATCHS
#    (mais rôle séparé domicile / extérieur)
# ============================================

df['home_recent_points5'] = (
    df.groupby('home_team')['home_points_match']
      .rolling(window=5, min_periods=1)
      .sum()
      .shift(1)
      .reset_index(level=0, drop=True)
)

df['away_recent_points5'] = (
    df.groupby('away_team')['away_points_match']
      .rolling(window=5, min_periods=1)
      .sum()
      .shift(1)
      .reset_index(level=0, drop=True)
)

# ============================================
# 5) REST DAYS : JOURS DE REPOS AVANT LE MATCH
# ============================================

df['prev_home_date'] = df.groupby('home_team')['date'].shift(1)
df['prev_away_date'] = df.groupby('away_team')['date'].shift(1)

df['rest_days_home'] = (df['date'] - df['prev_home_date']).dt.days
df['rest_days_away'] = (df['date'] - df['prev_away_date']).dt.days

# Pour le tout premier match d'une équipe, on met une valeur par défaut (7 jours)
df['rest_days_home'] = df['rest_days_home'].fillna(7)
df['rest_days_away'] = df['rest_days_away'].fillna(7)

# ============================================
# 6) BIG SIX 
# ============================================
print("➕ Adding Big 6 features...")

# ⚠️ Adapter ces noms à ceux de ton df si besoin
big6_teams = {
    'Arsenal FC',
    'Chelsea FC',
    'Liverpool FC',
    'Manchester City FC',
    'Manchester United FC',
    'Tottenham Hotspur FC'
}

df['home_big6'] = df['home_team'].isin(big6_teams).astype(int)
df['away_big6'] = df['away_team'].isin(big6_teams).astype(int)

print("✅ Big 6 features added: ['home_big6', 'away_big6']")
print(df[['home_team', 'away_team', 'home_big6', 'away_big6']].head(10))



# Nettoyer les colonnes temporaires
df.drop(columns=['prev_home_date', 'prev_away_date'], inplace=True)

print("✅ Extra features added:")
print([
    'home_points_cum', 'away_points_cum',
    'home_goal_diff_cum', 'away_goal_diff_cum',
    'home_recent_points5', 'away_recent_points5',
    'rest_days_home', 'rest_days_away'
])


➕ Adding extra league features (points, goal diff, recent form, rest days)...
➕ Adding Big 6 features...
✅ Big 6 features added: ['home_big6', 'away_big6']
                   home_team                   away_team  home_big6  away_big6
0                 Burnley FC          Manchester City FC          0          1
1                 Arsenal FC        Nottingham Forest FC          1          0
2            AFC Bournemouth          West Ham United FC          0          0
3  Brighton & Hove Albion FC               Luton Town FC          0          0
4                 Everton FC                   Fulham FC          0          0
5        Sheffield United FC           Crystal Palace FC          0          0
6        Newcastle United FC              Aston Villa FC          0          0
7               Brentford FC        Tottenham Hotspur FC          0          1
8                 Chelsea FC                Liverpool FC          1          1
9       Manchester United FC  Wolverhampton Wanderers 

In [13]:
print(df.columns.tolist())


['date', 'home_team', 'away_team', 'home_score', 'away_score', 'matchday', 'home_form', 'away_form', 'form_diff', 'home_goals_scored_avg', 'home_goals_conceded_avg', 'away_goals_scored_avg', 'away_goals_conceded_avg', 'offensive_strength_diff', 'defensive_strength_diff', 'result', 'home_team_value', 'away_team_value', 'value_diff', 'home_win_rate', 'away_win_rate', 'win_rate_diff', 'home_points_match', 'away_points_match', 'home_points_cum', 'away_points_cum', 'home_goal_diff_match', 'away_goal_diff_match', 'home_goal_diff_cum', 'away_goal_diff_cum', 'home_recent_points5', 'away_recent_points5', 'rest_days_home', 'rest_days_away', 'home_big6', 'away_big6']


In [14]:
print("💾 Saving dataset with features...\n")

# Colonnes de "metadata" (qu'on ne veut pas comme features)
metadata_cols = [
    'date', 'home_team', 'away_team',
    'home_score', 'away_score', 'matchday', 'result'
]

# Toutes les autres colonnes = features (y compris home_big6, away_big6, etc.)
feature_cols = [c for c in df.columns if c not in metadata_cols]

columns_to_save = metadata_cols + feature_cols

print("✅ Columns to save:")
print(columns_to_save)

df_features = df[columns_to_save].copy()

output_path = '../data/processed/matches_with_features.csv'
df_features.to_csv(output_path, index=False)

print(f"\n✅ Dataset saved to: {output_path}")
print(f"📦 Shape: {df_features.shape}")
print(f"📊 Features created: {len(feature_cols)} (excluding metadata)")


💾 Saving dataset with features...

✅ Columns to save:
['date', 'home_team', 'away_team', 'home_score', 'away_score', 'matchday', 'result', 'home_form', 'away_form', 'form_diff', 'home_goals_scored_avg', 'home_goals_conceded_avg', 'away_goals_scored_avg', 'away_goals_conceded_avg', 'offensive_strength_diff', 'defensive_strength_diff', 'home_team_value', 'away_team_value', 'value_diff', 'home_win_rate', 'away_win_rate', 'win_rate_diff', 'home_points_match', 'away_points_match', 'home_points_cum', 'away_points_cum', 'home_goal_diff_match', 'away_goal_diff_match', 'home_goal_diff_cum', 'away_goal_diff_cum', 'home_recent_points5', 'away_recent_points5', 'rest_days_home', 'rest_days_away', 'home_big6', 'away_big6']

✅ Dataset saved to: ../data/processed/matches_with_features.csv
📦 Shape: (760, 36)
📊 Features created: 29 (excluding metadata)
